# Python Review Project
First, we import necessary packages and set our working directory. We then load the proteomic, transcriptomic, and clinical CCRCC data, performing some initial reformatting on the proteomic dataset.

In [ ]:
# Import packages.
import os
import cptac
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

# Set working directory.
os.chdir('/mnt/c/analysis_data/')

# Download CCRCC dataset.
cptac.download(dataset="Ccrcc")

# Load dataset.
ccrcc = cptac.Ccrcc()

# Load protein, RNA, and clinical data.
protein_data = ccrcc.get_proteomics()
rna_data = ccrcc.get_transcriptomics()
clinical_data = ccrcc.get_clinical()

# Collapse column names in protein data and removes database IDs.
protein_data.columns = protein_data.columns.get_level_values(0)

Now, we perform all of our data preprocessing. First, we select only patients that are shared between all three datasets. We then filter any patients with missing tumor stage information. We also remove any genes with zero expression, as log-scaling this data would otherwise result in values of infinity. Subsequently, we impute expression levels for any proteins with missing expression information, as filtering results in the loss of a large proportion of our data. Finally, we log-scale the transcriptomic dataset.

In [2]:
# Select patients shared between all three dataframes.
shared_patients = np.intersect1d(rna_data.index, protein_data.index)

# Select genes shared between protein and RNA data.
shared_genes = np.intersect1d(rna_data.columns, protein_data.columns)

# Subset dataframes based on shared patients.
protein_shared = protein_data.loc[shared_patients, shared_genes]
rna_shared = rna_data.loc[shared_patients, shared_genes]
clinical_shared = clinical_data.loc[shared_patients, :]

# Filter any patients with NA tumor stage.
clinical_shared = clinical_shared.dropna(subset = "tumor_stage_pathological", axis = 0)
rna_shared = rna_shared.loc[clinical_shared.index, :]
protein_shared = protein_shared.loc[clinical_shared.index, :]

# Filter any genes with zero expression (for log scaling).
rna_shared = rna_shared.drop(rna_shared.columns[rna_shared.eq(0).any()], axis = 1)
protein_shared = protein_shared.loc[:, rna_shared.columns]

# Impute any proteins with NA expression.
imputer = KNNImputer(missing_values = np.nan)
transformed = imputer.fit_transform(protein_shared)
transformed = pd.DataFrame(transformed, index = protein_shared.index, columns = protein_shared.columns)
protein_shared.update(transformed)

# Log-scale RNA data.
rna_shared = np.log2(rna_shared)

We now determine differentially expressed genes/proteins between Stage I and Stage III patients for both the transcriptomic and proteomic data. We do this simplistically, taking the absolute difference of the mean gene expression across all patients in each tumor stage.

In [3]:
# Subset protein and RNA data into Stage I and Stage III patients.
clinical_s1 = clinical_shared.loc[clinical_shared["tumor_stage_pathological"] == "Stage I", :]
clinical_s3 = clinical_shared.loc[clinical_shared["tumor_stage_pathological"] == "Stage III", :]

# Calculate top five DEGs for RNA data.
rna_s1 = rna_shared[rna_shared.index.isin(clinical_s1.index)]
rna_s3 = rna_shared[rna_shared.index.isin(clinical_s3.index)]
rna_expr_diff = abs(rna_s1.mean() - rna_s3.mean())
rna_degs = rna_expr_diff.sort_values(ascending = False).head(5)
print(rna_degs.head)

# Calculate top five DEGs for protein data.
protein_s1 = protein_shared[protein_shared.index.isin(clinical_s1.index)]
protein_s3 = protein_shared[protein_shared.index.isin(clinical_s3.index)]
protein_expr_diff = abs(protein_s1.mean() - protein_s3.mean())
protein_degs = protein_expr_diff.sort_values(ascending = False).head(5)
print(protein_degs)

<bound method NDFrame.head of Name
AJAP1     1.738594
DPEP1     1.540969
GALNT5    1.513785
FABP7     1.352365
HP        1.284551
dtype: float64>
Name
FTL       0.862165
IGFLR1    0.707417
HBA2      0.589100
CMA1      0.583464
HBB       0.557303
dtype: float64


We then extract expression information for these differentially expressed genes/proteins and combine them into one dataframe. We also extract the tumor stage of each patient. These are our features and targets, respectively. To prepare for classifier training, we encode the tumor stages and scale the expression information.

In [4]:
# Subset DEGs for protein and RNA data into one dataframe.
features_deg = pd.concat([rna_shared.loc[:, rna_degs.index], protein_shared.loc[:, protein_degs.index]], axis = 1)

# Extract patient tumor stages.
targets_stage = clinical_shared[["tumor_stage_pathological"]]

# Encode patient tumor stages.
encoder = OrdinalEncoder()
targets_stage = encoder.fit_transform(targets_stage)

# Scale features.
scaler = StandardScaler()
features_deg = scaler.fit_transform(features_deg)

Now, we test four classifiers: K-Nearest Neighbors, Decision Tree, Multi-Layer Perceptron, and Naive-Bayes. We run each classifier ten times, performing a new 70/30 train-test split on each trial. The mean accuracy of each classifier across these ten trials is then output.

In [5]:
# Classifiers to test.
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    MLPClassifier(max_iter = 1000),
    GaussianNB()
]
classifiers_names = ['K-Nearest Neighbors', 'Decision Tree', 'Multi-Layer Perceptron', 'Naive-Bayes']

classifiers_acc = {
    0: [],
    1: [],
    2: [],
    3: [],
}

# Test each classifier and output mean accuracy across ten trials.
for i in range(len(classifiers)):
    accuracy = []
    classifier = classifiers[i]
    for j in range(10):
        X_train, X_test, y_train, y_test = train_test_split(features_deg, targets_stage.ravel(), train_size = 0.70)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        classifiers_acc[i].append(accuracy_score(y_test, y_pred))

# Print results.
print('\nAfter 10 simulations, the average accuracy for each classifier is as follows:')
for i in classifiers_acc:
    print(f'\t{classifiers_names[i]} : {round(np.mean(classifiers_acc[i]) * 100, ndigits = 2)}%')


After 10 simulations, the average accuracy for each classifier is as follows:
	K-Nearest Neighbors : 52.42%
	Decision Tree : 37.88%
	Multi-Layer Perceptron : 50.0%
	Naive-Bayes : 56.36%


Based on the above results, it appears that the Naive-Bayes classifier has the best predictive accuracy here. 